In [2]:
import os
import json
import random
import psycopg2
import mysql.connector
from faker import Faker
from datetime import datetime
from PIL import Image, ImageDraw
from kafka import KafkaProducer

# 📌 Initialiser Faker pour générer des données réalistes
fake = Faker()

# 📁 Dossiers pour stocker les fichiers
os.makedirs("data_lake/raw", exist_ok=True)
os.makedirs("data_lake/raw/images", exist_ok=True)
os.makedirs("data_lake/raw/logs", exist_ok=True)

# ======================================
# 📌 1. Génération de Données Relationnelles (PostgreSQL & MySQL)
# ======================================
def generate_sql_data():
    try:
        # 📌 Connexion PostgreSQL
        pg_conn = psycopg2.connect(database="ecommerce_pg", user="postgres", password="azerty143rose", host="localhost", port="5432")
        pg_cursor = pg_conn.cursor()
        pg_cursor.execute("CREATE TABLE IF NOT EXISTS clients (id SERIAL PRIMARY KEY, nom VARCHAR(255), email VARCHAR(255), pays VARCHAR(100));")

        # 📌 Connexion MySQL
        my_conn = mysql.connector.connect(user='logco', password='logCo@143', host='localhost', database='ecommerce_mysql')
        my_cursor = my_conn.cursor()
        my_cursor.execute("CREATE TABLE IF NOT EXISTS transactions (id INT AUTO_INCREMENT PRIMARY KEY, client_id INT, montant DECIMAL(10,2), date_achat DATE);")

        # 📌 Insérer des données
        for _ in range(101):
            name, email, country = fake.name(), fake.email(), fake.country()
            pg_cursor.execute("INSERT INTO clients (nom, email, pays) VALUES (%s, %s, %s);", (name, email, country))

            client_id, amount, date = random.randint(1, 10), round(random.uniform(10, 500), 2), fake.date()
            my_cursor.execute("INSERT INTO transactions (client_id, montant, date_achat) VALUES (%s, %s, %s);", (client_id, amount, date))

        pg_conn.commit()
        my_conn.commit()
        print("✅ Données SQL générées avec succès !")

    except Exception as e:
        print(f"⚠ Erreur SQL : {e}")

    finally:
        pg_cursor.close()
        pg_conn.close()
        my_cursor.close()
        my_conn.close()

# ======================================
# 📌 2. Génération de Données Semi-Structurées (CSV & JSON)
# ======================================
def generate_csv_json():
    # 📌 Générer un fichier CSV (Produits)
    with open("data_lake/raw/produits.csv", "w") as f:
        f.write("id,nom,prix,categorie\n")
        for i in range(1, 101):
            f.write(f"{i},{fake.word()},{round(random.uniform(5, 200), 2)},{fake.word()}\n")
    print("✅ Fichier produits.csv généré !")

    # 📌 Générer un fichier JSON (Commandes)
    commandes = [{"id": i, "produit": fake.word(), "quantité": random.randint(1, 5), "date": fake.date()} for i in range(10)]
    with open("data_lake/raw/commandes.json", "w") as f:
        json.dump(commandes, f, indent=4)
    print("✅ Fichier commandes.json généré !")

# ======================================
# 📌 3. Génération de Données Non Structurées (Texte & Logs)
# ======================================
def generate_text_logs():
    # 📌 Générer un fichier texte (Avis clients)
    with open("data_lake/raw/avis_clients.txt", "w") as f:
        for _ in range(101):
            f.write(fake.text() + "\n")
    print("✅ Fichier avis_clients.txt généré !")

    # 📌 Générer un fichier log (Logs serveur)
    with open("data_lake/raw/logs/serveur.log", "w") as f:
        for _ in range(101):
            log = f"{datetime.now()} - INFO - {fake.sentence()}"
            f.write(log + "\n")
    print("✅ Fichier serveur.log généré !")

# ======================================
# 📌 4. Génération d'Images (Non Structurées)
# ======================================
def generate_images():
    for i in range(50):
        img = Image.new("RGB", (100, 100), color=(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
        draw = ImageDraw.Draw(img)
        draw.text((10, 40), f"Prod {i+1}", fill=(255, 255, 255))
        img.save(f"data_lake/raw/images/produit_{i+1}.png")
    print("✅ Images produits générées !")

# ======================================
# 📌 5. Génération de Données en Streaming (Kafka)
# ======================================
def generate_kafka_data():
    try:
        producer = KafkaProducer(bootstrap_servers="localhost:9092")
        for _ in range(10):
            transaction = {
                "client_id": random.randint(1, 10),
                "montant": round(random.uniform(10, 500), 2),
                "date": fake.date()
            }
            producer.send("transactions", json.dumps(transaction).encode("utf-8"))
        print("✅ Messages envoyés à Kafka !")
    except Exception as e:
        print(f"⚠ Erreur Kafka : {e}")

# ======================================
# 📌 Exécuter toutes les fonctions
# ======================================
if __name__ == "__main__":
    generate_sql_data()
    generate_csv_json()
    generate_text_logs()
    generate_images()
    generate_kafka_data()


✅ Données SQL générées avec succès !
✅ Fichier produits.csv généré !
✅ Fichier commandes.json généré !
✅ Fichier avis_clients.txt généré !
✅ Fichier serveur.log généré !
✅ Images produits générées !
⚠ Erreur Kafka : NoBrokersAvailable
